# RF Python Serial

In [1]:
import sys
print (sys.version)
print (sys.version_info)

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [2]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']


## Main

Save the file to disk

In [71]:
%%writefile rfas.py
import pandas as pd
import numpy as np
import sys
from scipy.io import arff
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from time import time
t = time()

data = arff.loadarff(sys.argv[1])
df = pd.DataFrame(data[0])
df = df.replace(b'N', 0)
df = df.replace(b'Y', 1)
df['class'] = df['class'].str.decode('utf-8').fillna(df['class'])
y_train = df['class']
X_train = df.drop(columns=['class'])
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
df2 = pd.DataFrame(imp.fit_transform(X_train))
df2.columns = X_train.columns
df2.index = X_train.index
X_train = df2

datat = arff.loadarff(sys.argv[2])
df = pd.DataFrame(datat[0])
df = df.replace(b'N', 0)
df = df.replace(b'Y', 1)
df['class'] = df['class'].str.decode('utf-8').fillna(df['class'])
y_test = df['class']
X_test = df.drop(columns = ['class'])
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
df2 = pd.DataFrame(imp.fit_transform(X_test))
df2.columns = X_test.columns
df2.index = X_test.index
X_test = df2

clf = RandomForestClassifier(n_estimators = 100)
clf.fit(X_train, y_train)
y_pred_test  = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
accu = metrics.accuracy_score(y_train, y_pred_train, normalize = False)
trsi = y_train.size
perr = ((trsi - accu) / (trsi)) * 100
kapp = metrics.cohen_kappa_score(y_train, y_pred_train)
print(f'Trainset classification error is {perr:.2f}% ',
      f'of {trsi} (kappa: {kapp:.4f})')
accu = metrics.accuracy_score(y_test, y_pred_test, normalize = False)
trsi = y_test.size
perr = ((trsi - accu) / (trsi)) * 100
kapp = metrics.cohen_kappa_score(y_test, y_pred_test)
print(f' Testset classification error is {perr:.2f}% ',
      f'of {trsi} (kappa: {kapp:.4f})')

t = time() - t
print(f"T : {t:.4f} s")

Overwriting rfa.py


## Check

In [ ]:
! time python rfa.py datasets/asteroid-train.arff datasets/asteroid-test.arff

## Copy to scratch

In [3]:
%%bash
dir=/scratch${PWD#"/prj"}
cp rfas.py $dir

## Slurm script

In [4]:
%%writefile rfa_pyt_ser.srm
#!/bin/bash
# 1,0 UA partitions:
#   cpu,       96 h,    21-50 nodes, 4/24  tasks
#   cpu_dev,   20 min., 1-4   nodes, 1/1   tasks
#   cpu_small, 72 h,    1-20  nodes, 16/96 tasks
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --job-name rfs         # Job name
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes
#   NTASKS: 1, 4, 9, 16, 36, 49, 64, 81
echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
dir=/scratch${PWD#"/prj"}
cd $dir

# Executable config
EXEC=$dir/"python rafs.py datasets/asteroid-train.arff datasets/asteroid-test.arff"

# Start
echo '$ srun -n' $SLURM_NTASKS $EXEC
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting stc_pyt_seq.srm


### Run

In [ ]:
! sbatch --partition=cpu_dev rfa_pyt_ser.srm

In [ ]:
! squeue -n rfs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

In [ ]:
! cat /scratch${PWD#"/prj"}/slurm-xxxx.out

In [ ]:
! sbatch --partition=cpu_dev rfa_pyt_ser.srm

In [ ]:
! squeue -n rfs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

In [ ]:
! cat /scratch${PWD#"/prj"}/slurm-xxxx.out

In [ ]:
! sbatch --partition=cpu_dev rfa_pyt_ser.srm

In [ ]:
! squeue -n rfs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

In [ ]:
! cat /scratch${PWD#"/prj"}/slurm-xxxx.out